In [1]:
import os
import numpy as np
import pandas as pd
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras import Input
from tensorflow.keras.layers import Dense, MaxPooling2D, Conv2D,MaxPooling3D, Conv3D, Flatten
from tensorflow.keras.models import Sequential
from keras.models import load_model
from faceRecon import FaceExtractorMultithread, FaceExtractor

# Number of deepfakes and not deepfakes

In [12]:
from sklearn.utils import shuffle
from MetricsModule import TrainingMetrics


print('Loading data...')
fragments =[ pd.read_hdf(f'dataframes/CelebDB/dataframe{i}_1200videos.h5', key=f'df{i}') for i in range(6)]
df = pd.concat(fragments)
df = shuffle(df)


#count number of 1s and 0s in the label column
df['label'].value_counts()

Loading data...


label
0    19081
1     3531
Name: count, dtype: int64

# Whole module

In [4]:
from sklearn.utils import shuffle
import os
import numpy as np
import pandas as pd
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras import Input
from tensorflow.keras.layers import Dense, MaxPooling2D, Conv2D,MaxPooling3D, Conv3D, Flatten,Lambda, Dropout
from tensorflow.keras.models import Sequential
from keras.models import load_model
from faceRecon import FaceExtractorMultithread, FaceExtractor

route = 'P:\TFG\Datasets\dataframes_test'

print('Loading dataframes...')
fragments = []
for i in range(20):
    # Assuming each dataframe is stored as a separate table in the HDF5 file
    if i%10==0:
        print(f'{i} dataframes loaded')
    chunk = pd.read_hdf(f'{route}\dataframe{i}_FaceForensics.h5', key=f'df{i}')
    fragments.append(chunk)
#fragments = [pd.read_hdf(f'dataframes/CelebDB/dataframe{i}_600videos.h5', key=f'df{i}') for i in range(2)]#6
df = pd.concat(fragments)

print(df.describe())
print(df.dtypes)
print(df['label'].value_counts())

X = df.drop(['label'], axis = 1)
y = df['label']

print('Dividing dataset into train and test...')
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.2, random_state=42, stratify = y)
X_train = np.stack(X_train['face'], axis=0)
X_test = np.stack(X_test['face'], axis=0)


2024-02-23 20:37:43.369162: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-02-23 20:37:43.427763: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-02-23 20:37:43.427816: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-02-23 20:37:43.429680: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-02-23 20:37:43.439590: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-02-23 20:37:43.440698: I tensorflow/core/platform/cpu_feature_guard.cc:1

ModuleNotFoundError: No module named 'faceRecon'

In [13]:
print('Creating model...')
model = Sequential()
model.add(Input(shape=(200, 200, 3)))
model.add(Lambda(lambda x: x/255.0)) #normalizamos los valores de los pixeles
model.add(Conv2D(32, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Flatten())
model.add(Dropout(0.5))  # Dropout for regularization
model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.5))  # Dropout for regularization
model.add(Dense(1024, activation='relu'))
model.add(Dense(1, activation='softmax'))

model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])


Creating model...


In [14]:
from MetricsModule import TrainingMetrics
#prueba de que imprime las stats
metrics = TrainingMetrics(model,"P:\TFG\\results")
metrics.train(X_train, y_train, X_test, y_test, epochs=10)

Started training....
Epoch 1/10
902/902 [==============================] - 547s 597ms/step - loss: 0.2505 - accuracy: 0.2347 - val_loss: 0.1211 - val_accuracy: 0.2348
Epoch 2/10
902/902 [==============================] - 492s 546ms/step - loss: 0.0978 - accuracy: 0.2347 - val_loss: 0.0607 - val_accuracy: 0.2348
Epoch 3/10
902/902 [==============================] - 502s 557ms/step - loss: 0.0786 - accuracy: 0.2347 - val_loss: 0.0667 - val_accuracy: 0.2348
Epoch 4/10
902/902 [==============================] - 504s 559ms/step - loss: 0.0582 - accuracy: 0.2347 - val_loss: 0.0519 - val_accuracy: 0.2348
Epoch 5/10
383/902 [===========>..................] - ETA: 4:40 - loss: 0.0632 - accuracy: 0.2338

KeyboardInterrupt: 